Created 2022-12-31

Variational autoencoders are a play on the classic *autoencoder*. Whereas the original autoencoder maps input vectors onto latent "compressed" vectors, **variational autoencoders** map input vectors onto the parameters of a probability distribution instead. For example, a simple variational autoencoder taking in 1d values may map them onto $(\mu, \sigma)$, a 2d-vector representing the parameters of a 1d gaussian.

In [3]:
import tensorflow as tf
from IPython import display
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
import tensorflow_probability as tfp
import time

In [7]:
train_images.shape

(60000, 28, 28)

In [8]:
np.where()

<function numpy.where>

In [5]:
#We can disregard the data labels because our goal is not training a classifier model

(train_images, _), (test_images, _) = tf.keras.datasets.mnist.load_data()

In [18]:
'''
In this function, we first reshape the images to add an extra dimension.
Then, we scale the pixel values to be in [0,1] instead of [0,255]. Finally,
we binarize the image pixel values, so that any value at or below 0.5 becomes 0 while the rest 
become 1.
'''
def preprocess_images(images):
    images = images.reshape(images.shape[0], 28, 28, 1)
    images = images / 255
    images = np.where(images > 0.5, 1.0, 0.0).astype('float32')
    
    return images

train_images = preprocess_images(train_images)
test_images = preprocess_images(test_images)

In [19]:
train_size = 60000
batch_size = 32
test_size = 10000

In [24]:
'''
The tf.data.Dataset.from_tensor_slices function takes in a Tensor and creates a dataset with the items being 
indexed by the first dimension

'''

train_dataset = (tf.data.Dataset.from_tensor_slices(train_images).shuffle(buffer_size=train_size).batch(batch_size) )
test_dataset = (tf.data.Dataset.from_tensor_slices(test_images).shuffle(buffer_size=train_size).batch(batch_size))

In [ ]:
class ConvVAE(tf.keras.Model):
    
    def __init__(self, latent_dimension):
        super().__init__()
        self.latent_dimension = latent_dimension
        
        self.encoder = ([
            tf.keras.layers.InputLayer(input_shape=(28,28,1)), #https://stackoverflow.com/questions/45217973/what-is-the-advantage-of-using-an-inputlayer-or-an-input-in-a-keras-model-with
            tf.keras.layers.Conv2d(filters=32, #number of convolution kernels
                                   kernel_size=3,
                                   strides=(2,2),
                                   activation='relu'
                                  ),
            tf.keras.layers.Conv2d(filters=64,
                                  kernel_size=3,
                                  strides=(2,2),
                                  activation='relu'),
            tf.keras.layers.Flatten(),
            
            '''
            Here, the dense layer outputs a vector of length latent_dimension + latent_dimension.
            The first half of the entries will correspond to the means of gaussians, while the latter half of the entries
            will correspond to the log-variance of gaussians
            '''
            tf.keras.layers.Dense(latent_dimension + latent_dimension)
        
        ])
        
        self.decoder = ([
            tf.keras.layers.InputLayer(input_shape=(latent_dimension,)),
            tf.keras.layers.Dense(units=7*7*32, activation=tf.nn.relu),
            tf.keras.layers.Reshape(target_shape = (7,7,32)),
            tf.keras.layers.Conv2DTranspose(
                filters=64,
                kernel_size=3,
                strides=2,
                padding='same',
                activation='relu'
            ),
            tf.keras.layers.Conv2DTranspose(
                filters=32,
                kernel_size=3,
                strides=2,
                padding='same',
                activation='relu'
            )
            
            tf.keras.layers.Conv2DTranspose(
             filters=1,
            kernel_size=3,
                strides=1,
                padding='same'
            )
        
        ])
        
        
        @tf.function
        def sample

Sources:
- https://towardsdatascience.com/understanding-variational-autoencoders-vaes-f70510919f73
- https://www.tensorflow.org/tutorials/generative/cvae